# <span style="color:lightblue; font-weight:bold;">EXTRACCIÓ D'ENTITATS ANOMENADES</span>

## <span style="color:#ADD8E6; font-weight:bold;">Índex:</span>
### <span style="color:#FFFFFF;">1. [Predicció amb BIO](#bio)</span>
### <span style="color:#FFFFFF;">2. [Predicció amb IO](#io)</span>
### <span style="color:#ADD8E6;">3. [Predicció amb BIOES](#bioes)</span>
### <span style="color:#ADD8E6;">4. [Conclusions](#conclusions)</span>

## <span style="color:lightblue;"> Imports i Descarregues </span>

In [1]:
import nltk
import pycrfsuite
from nltk.corpus import conll2002
from nltk.tag import CRFTagger
from sklearn.metrics import accuracy_score

In [ ]:
nltk.download('punkt') # Tokenitzador
nltk.download('averaged_perceptron_tagger') # Etiquetador POS
nltk.download('maxent_ne_chunker') # Etiquetador Entitats Anomenades
nltk.download('words')
nltk.download('treebank')
nltk.download('conll2002')

## <span style="color:lightblue;"> Inicialització </span>

En aquesta secció obtenim els conjunts que utilitzarem més endavant per entrenar i avaluar els nostres models de reconeixement d'entitats anomenades. 
A més, definim les funcions que utilitzarem a posteriori per automatitzar procesos repetitius.

In [2]:
train_esp = conll2002.iob_sents('esp.train') # Train
testa_esp = conll2002.iob_sents('esp.testa') # Dev
testb_esp = conll2002.iob_sents('esp.testb') # Test

train_ned = conll2002.iob_sents('ned.train') # Train
testa_ned = conll2002.iob_sents('ned.testa') # Dev
testb_ned = conll2002.iob_sents('ned.testb') # Test

Definim les funcions per a obtenir diferents configuracions de representació dels nostres conjunts a entrenar i avaluar.

In [3]:
def obtenir_token_POS(fitxer):
    """
    Funció per convertir un text amb el token, POS i entitat per 
    cada element en cada frase en un text amb el token i el seu POS
    per cada element.
    """
    res = []
    for sentence in fitxer:
        frases = []
        for elem1, elem2, elem3 in sentence:
            frases.append((elem1, elem2))
        res.append(frases)
    return res

# Fem prediccions i mirem l'accuracy
def obtenir_token(fitxer):
    """
    Funció per convertir un text amb el token, POS i entitat 
    per cada element en cada frase en un text amb només el token
    per cada element.
    """
    res = []
    for sentence in fitxer:
        frases = []
        for elem1, elem2, elem3 in sentence:
            frases.append(elem1)
        res.append(frases)
    return res


def obtenir_token_entity(fitxer):
    """
    Funció per convertir un text amb el token, POS i entitat per cada 
    element en cada frase en un text amb el token i la seva entitat per 
    cada element.
    """
    res = []
    for sentence in fitxer:
        frases = []
        for elem1, elem2, elem3 in sentence:
            frases.append((elem1, elem3))
        res.append(frases)
    return res

### <span style="color:lightblue;"> Classe FeatureExtractor personalitzada </span>

<span> Features que es tenen en compte:
<ul>
    <li>Paraula actual</li>
    <li>Si comença en majúscula</li>
    <li>Si té signe de puntuació</li>
    <li>Si té números</li>
    <li>Prefixos fins a longitud 3</li>
    <li>Sufixos fins a longitud 3</li>
    <li>Paraules prèvies i posteriors amb POS</li>
    <li>POS-tags</li>
    <li>Longitud de la paraula</li>
</ul>
</span>

In [4]:
import unicodedata
import re

class FeatureExtractor:
    
    """
    Aquesta classe conté el mètode per calcular les features
    que s'utilitzaran per entrenar el model més endavant.
    """
    
    def __init__(self, use_basic_features=False, use_prefix_suffix_features=False, use_context_features=False, pattern = r'\d+', model_POS = None):
        self.use_basic_features = use_basic_features
        self.use_prefix_suffix_features = use_prefix_suffix_features
        self.use_context_features = use_context_features
        self._pattern = pattern
        self.model_POS = model_POS
        
    def __str__(self):
        features = []
        if self.use_basic_features:
            features.append("use_basic_features=True")
        if self.use_prefix_suffix_features:
            features.append("use_prefix_suffix_features=True")
        if self.use_context_features:
            features.append("use_context_features=True")
        return f"FeatureExtractor({', '.join(features)})"
        
    def _get_features(self, tokens, idx):
        """
        Extract basic features about this word including
            - Current word
            - is it capitalized?
            - Does it have punctuation?
            - Does it have a number?
            - Preffixes up to length 3
            - Suffixes up to length 3
            - paraules prèvies i posteriors amb POS
            - POS-tags
            - longitud

        Note that : we might include feature over previous word, next word etc.

        :return: a list which contains the features
        :rtype: list(str)
        """
            
        token = tokens[idx]
        
        feature_list = []
        
        if self.use_basic_features:
            # Capitalization
            if token[0].isupper():
                feature_list.append("CAPITALIZATION")

            # Number
            if re.search(self._pattern, token) is not None:
                feature_list.append("HAS_NUM")

            # Punctuation
            punc_cat = {"Pc", "Pd", "Ps", "Pe", "Pi", "Pf", "Po"}
            if all(unicodedata.category(x) in punc_cat for x in token):
                feature_list.append("PUNCTUATION")

                    
        if self.use_prefix_suffix_features:
            # preffix up to length 3
            if len(token) > 1:
                feature_list.append("PRE_" + token[:1])
            if len(token) > 2:
                feature_list.append("PRE_" + token[:2])
            if len(token) > 3:
                feature_list.append("PRE_" + token[:3])

            # Suffix up to length 3
            if len(token) > 1:
                feature_list.append("SUF_" + token[-1:])
            if len(token) > 2:
                feature_list.append("SUF_" + token[-2:])
            if len(token) > 3:
                feature_list.append("SUF_" + token[-3:])
    
        
        if self.use_context_features:
            # POS_tags
            POS = self.model_POS.tag(tokens)
                
            # Paraules prèvies amb POS
            if idx > 0:
                feature_list.append("anterior1_" + tokens[idx-1] + "_" + POS[idx-1][1])
            if idx > 1:
                feature_list.append("anterior2_" + tokens[idx-2] + "_" + POS[idx-2][1])
                
            # Paraules posteriors amb POS
            if idx < (len(tokens)-1):
                feature_list.append("posterior1_" + tokens[idx+1] + "_" + POS[idx+1][1])
            if idx < (len(tokens)-2):
                feature_list.append("posterior2_" + tokens[idx+2] + "_" + POS[idx+2][1])

            feature_list.append("WORD_" + token + "_" + POS[idx][1])
            
        
        if not self.use_context_features:
            feature_list.append("WORD_" + token)
            
        
        return feature_list

### <span style="color:lightblue;"> Mètriques d'avaluació </span>

In [5]:
def calcular_precisio(entitats_referencia, entitats_predites):
    # Calcular el número de entidades correctamente extraídas
    entitats_correctes = entitats_referencia.intersection(entitats_predites)
    
    # Calcular la precisión
    if len(entitats_predites) > 0:
        precisio = len(entitats_correctes) / len(entitats_predites)
    else:
        precisio = 0.0
    
    return precisio


def calcular_recall(entitats_referencia, entitats_predites):
    # Calcular el número de entidades correctamente extraídas
    entitats_correctes = entitats_referencia.intersection(entitats_predites)
    
    # Calcular recall
    if len(entitats_referencia) > 0:
        recall = len(entitats_correctes) / len(entitats_referencia)
    else:
        recall = 0.0
    
    return recall

def calcular_f1_score(precisio, recall):
    # Calcular el F1-score
    if (precisio + recall) > 0:
        f1_score = 2 * (precisio * recall) / (precisio + recall)
    else:
        f1_score = 0.0
    
    return f1_score

def resultats(predicted_BIO, testa_esp_BIO_tag, obtain_entity):

    # Obtener los conjuntos de entidades de referencia y extraídas
    entitats_referencia = obtain_entity(testa_esp_BIO_tag)  # Conjunto de entidades etiquetadas manualmente como referencia
    entitats_predites =  obtain_entity(predicted_BIO) # Obtener conjuntos de entidades predichas

    # Calcular la precisión
    precisio = calcular_precisio(entitats_referencia, entitats_predites)

    # Calcular la exhaustividad
    recall = calcular_recall(entitats_referencia, entitats_predites)

    # Calcular el F1-score
    f1_score = calcular_f1_score(precisio, recall)

    print("Precisió:", precisio)
    print("Recall:", recall)
    print("F1-score:", f1_score)


## <span style="color:lightblue; font-weight:bold;">Predicció amb BIO</span> <a id="bio"></a>

In [6]:
def obtenir_entitats_amb_posicions_BIO(fitxer_BIO_tag):
    """
    Funció per agrupar en sets les entitats anomenades, l'índex on comença i l'índex on acaba,
    i la classe de la entitat.
    
    Argument: un text amb una tupla (amb el token i la seva entitat) per cada element en cada frase.
    """
    
    entitats_amb_posicions = set()

    for sentence_index, sentence in enumerate(fitxer_BIO_tag):
        ent = []
        name = None
        start_pos = None  # Posición de inicio de la entidad actual
        prev_tag = None  # Almacenar la etiqueta del token anterior

        for token_index, token in enumerate(sentence):
            word, tag = token
            #word = word[0]

            if tag.startswith('B-'):
                # Si hay una entidad anterior, la agregamos a la lista de entidades
                if ent:
                    end_pos = token_index - 1  # La posición de fin es el token anterior
                    entitats_amb_posicions.add((tuple(ent), (start_pos, end_pos), name))
                # Creamos una nueva entidad con la palabra actual
                ent = [word]
                # Obtenemos el tipo de entidad
                name = tag.split('-')[1]
                start_pos = token_index  # La posición de inicio es el token actual
                prev_tag = tag  # Actualizamos la etiqueta del token anterior
            elif tag.startswith('I-'):
                # Solo agregamos la palabra actual si el token anterior tiene etiqueta I- o B-
                if prev_tag:
                    ent.append(word)
                    prev_tag = tag  # Actualizamos la etiqueta del token anterior
            elif tag == 'O' and ent:
                # Si encontramos una etiqueta 'O' y hay una entidad en curso, la agregamos a la lista de entidades
                end_pos = token_index - 1  # La posición de fin es el token anterior
                entitats_amb_posicions.add((tuple(ent), (start_pos, end_pos), name))
                # Reiniciamos la lista de la entidad actual
                ent = []
                prev_tag = None  # Reiniciamos la etiqueta del token anterior

        # Agregamos la última entidad si la hay
        if ent:
            end_pos = len(sentence) - 1  # La posición de fin es el último token de la oración
            entitats_amb_posicions.add((tuple(ent), (start_pos, end_pos)))

    return entitats_amb_posicions

#### ESP

Comencem entrenant un model CRFTagger per a la predicció de les etiquetes POS, ja que serà utilitzat posteriorment per a calcular els features necessaris en el procés d'extracció d'entitats anomenades. Inicialment, vam considerar utilitzar les etiquetes POS existents en els textos com a dades d'entrada, però vam enfrontar-nos a un error durant el procés d'entrenament del model CRFTagger. Aquest error indicava que el model esperava rebre només un element amb la seva etiqueta, en comptes de dos elements amb una etiqueta, ja que estàvem passant una tupla que contenia tant el token com el seu POS, juntament amb l'etiqueta de l'entitat. Per tant, per evitar aquest problema, vam optar per entrenar un model separat per a la predicció de les etiquetes POS, que posteriorment seran utilitzades com a característiques addicionals en el procés d'entrenament per a l'extracció d'entitats anomenades.

Veiem que l'accuracy del model és prou elevada per tenir-lo en compte i utilitzar-lo més endavant.

In [7]:
model_tagger_POS_esp = CRFTagger()

# Entrenem el model per predir els POS que corresponen a cada token

train_esp_pos_tag = obtenir_token_POS(train_esp)
    
model_tagger_POS_esp.train(train_esp_pos_tag, 'model_POS_esp.crf.tagger')    


testa_esp_pre_tag = obtenir_token(testa_esp)
    
predicted = model_tagger_POS_esp.tag_sents(testa_esp_pre_tag)

predictions = [elem[1] for sentence in predicted for elem in sentence]
real_label = [elem[1] for sentence in testa_esp for elem in sentence]

print(accuracy_score(predictions, real_label))

0.9447121289420479


A continuació, realitzem experiments amb diverses features per determinar quines proporcionen un rendiment òptim pel model amb etiquetatge BIO, i en textos escrits en espanyol. A més, aquests experiments els executem en el conjunt de proves "testa", que considerem com a conjunt de validació per ajustar i identificar les millors característiques.

In [8]:
param_combinations_esp = [
    None,
    FeatureExtractor(),
    FeatureExtractor(True, model_POS=model_tagger_POS_esp),
    FeatureExtractor(True, True, model_POS=model_tagger_POS_esp),
    FeatureExtractor(True, True, True, model_POS=model_tagger_POS_esp)
]

train_esp_BIO = obtenir_token_entity(train_esp)

testa_esp_real = obtenir_token_entity(testa_esp)

def model_entrenament(train_esp_BIO_tag, extractor, testa_esp_pre_tag):
    if extractor == None:
        model_BIO = CRFTagger()
        model_BIO.train(train_esp_BIO_tag, 'model_BIO.crf.tagger')
        
        predicted_BIO = model_BIO.tag_sents(testa_esp_pre_tag)
    
    else:
        model_BIO = CRFTagger(feature_func=extractor._get_features)
        model_BIO.train(train_esp_BIO_tag, 'model_BIO.crf.tagger')
        
        predicted_BIO = model_BIO.tag_sents(testa_esp_pre_tag)
    
    return resultats(predicted_BIO, testa_esp_real, obtenir_entitats_amb_posicions_BIO)

for param in param_combinations_esp:
    print(f'Model: {str(param)}') if param != None else print(f'Model: Features per defecte de CRFTagger')
    model_entrenament(train_esp_BIO, param, testa_esp_pre_tag)
    print('-'*50)
    

Model: None
Precisió: 0.6927890345649583
Recall: 0.643687707641196
F1-score: 0.6673363949483353
--------------------------------------------------
Model: FeatureExtractor()
Precisió: 0.718299164768413
Recall: 0.2619047619047619
F1-score: 0.3838506796510448
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True)
Precisió: 0.6447638603696099
Recall: 0.6085271317829457
F1-score: 0.626121635094716
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True)
Precisió: 0.7015675835551612
Recall: 0.6566998892580288
F1-score: 0.6783926783926784
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True, use_context_features=True)
Precisió: 0.7475160724722385
Recall: 0.7081949058693244
F1-score: 0.7273244242251918
--------------------------------------------------


Observem que els millors resultats es donen amb el model que usa els 3 'features' diferents i, per tant, usarem aquest model per a poder identificar les entitats correctament en el conjunt de test que no hem utilitzat fins ara, el 'testb'.

In [10]:
feature_extractor = FeatureExtractor(True, True, True, model_POS=model_tagger_POS_esp)

testb_esp_real = obtenir_token_entity(testb_esp)
testb_esp_pre_tag = obtenir_token(testb_esp)

model_BIO_esp = CRFTagger(feature_func=feature_extractor._get_features)
model_BIO_esp.train(train_esp_BIO, 'model_BIO.crf.tagger')
    
predicted_BIO = model_BIO_esp.tag_sents(testb_esp_pre_tag)

resultats(predicted_BIO, testb_esp_real, obtenir_entitats_amb_posicions_BIO)


Precisió: 0.7869019341703427
Recall: 0.7645895153313551
F1-score: 0.7755852842809364


#### NED

Entrenem el model de CRF per predir els POS tags però ara pel nederlandés.

In [11]:
model_tagger_POS_ned = CRFTagger()

# Entrenem el model per predir els POS que corresponen a cada token
train_ned_pos_tag = obtenir_token_POS(train_ned)
    
model_tagger_POS_ned.train(train_ned_pos_tag, 'model_POS_ned.crf.tagger')


# Fem prediccions i mirem l'accuracy
testa_ned_pre_tag = obtenir_token(testa_ned)
    
predicted = model_tagger_POS_ned.tag_sents(testa_ned_pre_tag)

predictions = [elem[1] for sentence in predicted for elem in sentence]
real_label = [elem[1] for sentence in testa_ned for elem in sentence]

print(accuracy_score(predictions, real_label))

0.940191577997718


Experimentació amb diferents 'features' per a veure quins són els més adequats pel model amb 'BIO' i la llengua neerlandesa.

In [12]:
param_combinations_ned = [
    None,
    FeatureExtractor(),
    FeatureExtractor(True, model_POS=model_tagger_POS_ned),
    FeatureExtractor(True, True, model_POS=model_tagger_POS_ned),
    FeatureExtractor(True, True, True, model_POS=model_tagger_POS_ned)
]

train_ned_BIO = obtenir_token_entity(train_ned)

testa_ned_real = obtenir_token_entity(testa_ned)

def model_entrenament(train_BIO_tag, extractor, testa_pre_tag):
    if extractor == None:
        model_BIO = CRFTagger()
        model_BIO.train(train_BIO_tag, 'model_BIO_ned.crf.tagger')
        
        predicted_BIO = model_BIO.tag_sents(testa_pre_tag)
    
    else:    
        model_BIO = CRFTagger(feature_func=extractor._get_features)
        model_BIO.train(train_BIO_tag, 'model_BIO_ned.crf.tagger')
        
        predicted_BIO = model_BIO.tag_sents(testa_pre_tag)
    
    return resultats(predicted_BIO, testa_ned_real, obtenir_entitats_amb_posicions_BIO)

for param in param_combinations_ned:
    print(f'Model: {str(param)}') if param != None else print(f'Model: Features per defecte de CRFTagger')
    model_entrenament(train_ned_BIO, param, testa_ned_pre_tag)
    print('-'*50)

Model: None
Precisió: 0.6441908713692946
Recall: 0.565059144676979
F1-score: 0.602035870092099
--------------------------------------------------
Model: FeatureExtractor()
Precisió: 0.762114537444934
Recall: 0.15741583257506825
F1-score: 0.2609351432880845
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True)
Precisió: 0.655096011816839
Recall: 0.4035486806187443
F1-score: 0.49943693693693697
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True)
Precisió: 0.6853182751540041
Recall: 0.6073703366696998
F1-score: 0.6439942112879885
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True, use_context_features=True)
Precisió: 0.7082306554953179
Recall: 0.6537761601455869
F1-score: 0.6799148332150462
--------------------------------------------------


Observem que els millors resultats es donen amb el model que usa els 3 'features' diferents i, per tant, usarem aquest model per a poder identificar les entitats correctament.

In [13]:
feature_extractor = FeatureExtractor(True, True, True, model_POS=model_tagger_POS_ned)

testb_ned_real = obtenir_token_entity(testb_ned)
testb_ned_pre_tag = obtenir_token(testb_ned)

model_BIO_ned = CRFTagger(feature_func=feature_extractor._get_features)
model_BIO_ned.train(train_ned_BIO, 'model_BIO_ned.crf.tagger')
    
predicted_BIO = model_BIO_ned.tag_sents(testb_ned_pre_tag)

resultats(predicted_BIO, testb_ned_real, obtenir_entitats_amb_posicions_BIO)

Precisió: 0.7402511566424322
Recall: 0.6860643185298622
F1-score: 0.7121284374503258


## <span style="color:lightblue; font-weight:bold;">Predicció amb IO</span> <a id="io"></a>

Definim una funció per passar de l'etiquetatge 'BIO' a l'etiquetatge 'IO'.

In [14]:
def convert_to_io(train_data_bio):
    """
    Función para convertir datos de formato BIO a formato IO.
    Argumento:
    train_data_bio: una lista de frases, donde cada frase es una lista de tuplas (palabra, etiqueta POS, etiqueta BIO).
    Retorna:
    Una lista de frases en formato IO, donde cada frase es una lista de tuplas (palabra, etiqueta POS, etiqueta IO).
    """
    train_data_io = []
    for sentence in train_data_bio:
        io_tags = []
        for word, pos_tag, bio_tag in sentence:
            if bio_tag == 'O':
                io_tags.append('O')
            elif bio_tag.startswith('B-'):
                io_tags.append('I' + bio_tag[1:])
            else:
                io_tags.append(bio_tag)
        
        train_data_io.append([(word, pos_tag, io_tag) for (word, pos_tag, bio_tag), io_tag in zip(sentence, io_tags)])
    return train_data_io

Aquesta funció agrupa les entitats anomenades amb les seves posicions d'inici i fi, així com la seva classe, en un conjunt de tuples. Cada tuple conté tres elements: la llista de tokens que formen l'entitat, una tupla amb les posicions d'inici i fi (els índexs) de l'entitat dins del text, i la classe de l'entitat.

In [15]:
def obtenir_entitats_amb_posicions_IO(fitxer_IO_tag):
    """
    Función para agrupar en sets las entidades nombradas, el índice donde empieza y el índice donde acaba,
    y la clase de la entidad.
    
    Argumento: un texto con una tupla (con el token y su entidad) para cada elemento en cada frase.
    """
    
    entitats_amb_posicions = set()

    for sentence_index, sentence in enumerate(fitxer_IO_tag):
        ent = []
        name = None
        start_pos = None  # Posición de inicio de la entidad actual
        prev_tag = None  # Almacenar la etiqueta del token anterior

        for token_index, token in enumerate(sentence):
            word, tag = token

            if tag.startswith('I-'):
                # Si la etiqueta es 'I-' y el token anterior es 'O', comenzamos una nueva entidad
                if not prev_tag:
                    # Si hay una entidad anterior, la agregamos a la lista de entidades
                    if ent:
                        end_pos = token_index - 1  # La posición de fin es el token anterior
                        entitats_amb_posicions.add((tuple(ent), (start_pos, end_pos), name))
                    # Creamos una nueva entidad con la palabra actual
                    ent = [word]
                    # Obtenemos el tipo de entidad
                    name = tag.split('-')[1]
                    start_pos = token_index  # La posición de inicio es el token actual
                else:
                    # Si el token anterior es 'I-', agregamos la palabra actual a la entidad en curso
                    ent.append(word)
                prev_tag = tag  # Actualizamos la etiqueta del token anterior
            
            elif tag == 'O' and ent:
                # Si encontramos una etiqueta 'O' y hay una entidad en curso, la agregamos a la lista de entidades
                end_pos = token_index - 1  # La posición de fin es el token anterior
                entitats_amb_posicions.add((tuple(ent), (start_pos, end_pos), name))
                # Reiniciamos la lista de la entidad actual
                ent = []
                prev_tag = None  # Reiniciamos la etiqueta del token anterior

        # Agregamos la última entidad si la hay
        if ent:
            end_pos = len(sentence) - 1  # La posición de fin es el último token de la oración
            entitats_amb_posicions.add((tuple(ent), (start_pos, end_pos), name))

    return entitats_amb_posicions


#### ESP

Experimentació amb diferents 'features' per a veure quins són els més adequats pel model amb 'IO' i la llengua espanyola.

In [16]:
train_esp_io = convert_to_io(train_esp)
train_esp_IO = obtenir_token_entity(train_esp_io)

testa_esp_io = convert_to_io(testa_esp)
testa_esp_real = obtenir_token_entity(testa_esp_io)

def model_entrenament(train_tag, extractor, testa_pre_tag):
    if extractor == None:
        model_IO = CRFTagger()
        model_IO.train(train_tag, 'model_IO.crf.tagger')
        
        predicted_IO = model_IO.tag_sents(testa_pre_tag)
        
    else:
        model_IO = CRFTagger(feature_func=extractor._get_features)
        model_IO.train(train_tag, 'model_IO.crf.tagger')
        
        predicted_IO = model_IO.tag_sents(testa_pre_tag)
        
    return resultats(predicted_IO, testa_esp_real, obtenir_entitats_amb_posicions_IO)

for param in param_combinations_esp: # definida primerament en la predicció del BIO
    print(f'Model: {str(param)}') if param != None else print(f'Model: Features per defecte de CRFTagger')
    model_entrenament(train_esp_IO, param, testa_esp_pre_tag)
    print('-'*50)

Modelo: None
Precisió: 0.6720413751140858
Recall: 0.6208544125913434
F1-score: 0.6454346238130021
--------------------------------------------------
Modelo: FeatureExtractor()
Precisió: 0.6519083969465649
Recall: 0.24002248454187747
F1-score: 0.35086277732128185
--------------------------------------------------
Modelo: FeatureExtractor(use_basic_features=True)
Precisió: 0.6396209653538644
Recall: 0.6070826306913997
F1-score: 0.622927180966114
--------------------------------------------------
Modelo: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True)
Precisió: 0.6832579185520362
Recall: 0.636593591905565
F1-score: 0.6591008293321694
--------------------------------------------------
Modelo: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True, use_context_features=True)
Precisió: 0.738569753810082
Recall: 0.7082630691399663
F1-score: 0.7230989956958392
--------------------------------------------------


Observem que els millors resultats es donen amb el model que usa els 3 'features' diferents, com en el cas anterior. Farem servir el model que utilitza aquestes 3 'features' per a poder identificar les entitats correctament.

In [18]:
feature_extractor = FeatureExtractor(True, True, True, model_POS=model_tagger_POS_esp)

testb_esp_io = convert_to_io(testb_esp)
testb_esp_real = obtenir_token_entity(testb_esp_io)

testb_esp_pre_tag = obtenir_token(testb_esp)

model_IO_esp = CRFTagger(feature_func=feature_extractor._get_features)
model_IO_esp.train(train_esp_IO, 'model_IO.crf.tagger')
    
predicted_IO = model_IO_esp.tag_sents(testb_esp_pre_tag)

resultats(predicted_IO, testb_esp_real, obtenir_entitats_amb_posicions_IO)


Precisió: 0.779678412589805
Recall: 0.7553861451773285
F1-score: 0.7673400673400673


#### NED

Experimentació amb diferents 'features' per a veure quins són els més adequats pel model amb 'IO' i la llengua neerlandesa.

In [19]:
train_ned_io = convert_to_io(train_ned)
train_ned_IO = obtenir_token_entity(train_ned_io)

testa_ned_io = convert_to_io(testa_ned)
testa_ned_real = obtenir_token_entity(testa_ned_io)

def model_entrenament(train_tag, extractor, testa_pre_tag):
    if extractor == None:
        model_IO = CRFTagger()
        model_IO.train(train_tag, 'model_io_ned.crf.tagger')
        
        predicted_IO = model_IO.tag_sents(testa_pre_tag)
        
    else:
        model_IO = CRFTagger(feature_func=extractor._get_features)
        model_IO.train(train_tag, 'model_io_ned.crf.tagger')
        
        predicted_IO = model_IO.tag_sents(testa_pre_tag)
        
    return resultats(predicted_IO, testa_ned_real, obtenir_entitats_amb_posicions_IO)

for param in param_combinations_ned: # definida primerament en la predicció del BIO
    print(f'Model: {str(param)}') if param != None else print(f'Model: Features per defecte de CRFTagger')
    model_entrenament(train_ned_IO, param, testa_ned_pre_tag)
    print('-'*50)

Modelo: Features per defecte de CRFTagger
Precisió: 0.6347177848775293
Recall: 0.5614696184644371
F1-score: 0.5958510372406899
--------------------------------------------------
Modelo: FeatureExtractor()
Precisió: 0.7369727047146402
Recall: 0.13989637305699482
F1-score: 0.2351543942992874
--------------------------------------------------
Modelo: FeatureExtractor(use_basic_features=True)
Precisió: 0.623082542001461
Recall: 0.401789919924635
F1-score: 0.4885452462772051
--------------------------------------------------
Modelo: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True)
Precisió: 0.6628211851074987
Recall: 0.5953838907206783
F1-score: 0.6272952853598015
--------------------------------------------------
Modelo: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True, use_context_features=True)
Precisió: 0.7160931174089069
Recall: 0.6665096561469619
F1-score: 0.6904122956818737
--------------------------------------------------


En aquesta ocasió tornem a observar que els millors resultats es donen amb el model que usa els 3 'features' diferents, per tant, farem servir el model que utilitza aquestes 3 'features' per a poder identificar les entitats correctament.

In [20]:
feature_extractor = FeatureExtractor(True, True, True, model_POS=model_tagger_POS_ned)

testb_ned_io = convert_to_io(testb_ned)
testb_ned_real = obtenir_token_entity(testb_ned_io)

testb_ned_pre_tag = obtenir_token(testb_ned)

model_IO_ned = CRFTagger(feature_func=feature_extractor._get_features)
model_IO_ned.train(train_ned_IO, 'model_io_ned.crf.tagger')
    
predicted_IO = model_IO_ned.tag_sents(testb_ned_pre_tag)

resultats(predicted_IO, testb_ned_real, obtenir_entitats_amb_posicions_IO)

Precisió: 0.7119216480918609
Recall: 0.670057215511761
F1-score: 0.6903553299492386


## <span style="color:lightblue; font-weight:bold;">Predicció amb BIOES</span> <a id="bioes"></a>

In [21]:
def convert_to_bioes(train_data_bio):
    """
    Función para convertir datos de formato BIO a formato BIOES.
    Argumento:
    train_data_bio: una lista de frases, donde cada frase es una lista de tuplas (palabra, etiqueta POS, etiqueta BIO).
    Retorna:
    Una lista de frases en formato BIOES, donde cada frase es una lista de tuplas (palabra, etiqueta POS, etiqueta BIOES).
    """
    train_data_bioes = []
    for sentence in train_data_bio:
        bioes_tags = []
        for i, (word, pos_tag, bio_tag) in enumerate(sentence):
            if bio_tag == 'O':
                bioes_tags.append('O')
            elif bio_tag.startswith('B-'):
                if i == len(sentence) - 1 or sentence[i + 1][2] != 'I' + bio_tag[1:]:
                    bioes_tags.append('S' + bio_tag[1:])  # Single
                else:
                    bioes_tags.append('B' + bio_tag[1:])  # Begin
            elif bio_tag.startswith('I-'):
                if i == len(sentence) - 1 or sentence[i + 1][2] != 'I' + bio_tag[1:]:
                    bioes_tags.append('E' + bio_tag[1:])  # End
                else:
                    bioes_tags.append('I' + bio_tag[1:])  # Inside
            else:
                raise ValueError("Etiqueta BIO incorrecta: {}".format(bio_tag))
        
        train_data_bioes.append([(word, pos_tag, bioes_tag) for (word, pos_tag, bio_tag), bioes_tag in zip(sentence, bioes_tags)])
    return train_data_bioes

In [22]:
def obtenir_entitats_amb_posicions_bioes(train_data_bioes):
    """
    Función para agrupar en sets las entidades nombradas, el índice donde empieza y el índice donde acaba,
    y la clase de la entidad.
    
    Argumento: un texto con una tupla (con el token y su entidad) para cada elemento en cada frase.
    """
        
    entitats_amb_posicions = set()

    for sentence_index, sentence in enumerate(train_data_bioes):
        ent = []
        name = None
        start_pos = None  # Posición de inicio de la entidad actual

        for token_index, (word, bioes_tag) in enumerate(sentence):
            if bioes_tag != 'O':
                if bioes_tag.startswith('B-') or bioes_tag.startswith('S-'):
                    # Si hay una entidad anterior, la agregamos a la lista de entidades
                    if ent:
                        end_pos = token_index - 1  # La posición de fin es el token anterior
                        entitats_amb_posicions.add((tuple(ent), (start_pos, end_pos), name))
                    # Creamos una nueva entidad con la palabra actual
                    ent = [word]
                    # Obtenemos el tipo de entidad
                    name = bioes_tag.split('-')[1]
                    start_pos = token_index  # La posición de inicio es el token actual
                elif bioes_tag.startswith('I-') or bioes_tag.startswith('E-'):
                    ent.append(word)
            elif ent:
                # Si encontramos una etiqueta 'O' y hay una entidad en curso, la agregamos a la lista de entidades
                end_pos = token_index - 1  # La posición de fin es el token anterior
                entitats_amb_posicions.add((tuple(ent), (start_pos, end_pos), name))
                # Reiniciamos la lista de la entidad actual
                ent = []

        # Agregamos la última entidad si la hay
        if ent:
            end_pos = len(sentence) - 1  # La posición de fin es el último token de la oración
            entitats_amb_posicions.add((tuple(ent), (start_pos, end_pos), name))

    return entitats_amb_posicions


#### ESP

Experimentació amb diferents 'features' per a veure quins són els més adequats pel model amb 'BIO' i la llengua espanyola.

In [23]:
train_esp_bioes = convert_to_bioes(train_esp)
train_esp_BIOES = obtenir_token_entity(train_esp_bioes)

testa_esp_bioes = convert_to_bioes(testa_esp)
testa_esp_real = obtenir_token_entity(testa_esp_bioes)

def model_entrenament(train_tag, extractor, testa_pre_tag):
    if extractor == None:
        model_BIOES = CRFTagger()
        model_BIOES.train(train_tag, 'model_BIOES_esp.crf.tagger')
        
        predicted_BIOES = model_BIOES.tag_sents(testa_pre_tag)
        
    else:
        model_BIOES = CRFTagger(feature_func=extractor._get_features)
        model_BIOES.train(train_tag, 'model_BIOES_esp.crf.tagger')
        
        predicted_BIOES = model_BIOES.tag_sents(testa_pre_tag)
        
    return resultats(predicted_BIOES, testa_esp_real, obtenir_entitats_amb_posicions_bioes)

for param in param_combinations_esp: # definida primerament en la predicció del BIO
    print(f'Model: {str(param)}') if param != None else print(f'Model: Features per defecte de CRFTagger')
    model_entrenament(train_esp_BIOES, param, testa_esp_pre_tag)
    print('-'*50)

Modelo: Features per defecte de CRFTagger
Precisió: 0.6887340301974448
Recall: 0.6565181289786881
F1-score: 0.6722403287515941
--------------------------------------------------
Modelo: FeatureExtractor()
Precisió: 0.7877838684416602
Recall: 0.2784389703847218
F1-score: 0.4114519427402863
--------------------------------------------------
Modelo: FeatureExtractor(use_basic_features=True)
Precisió: 0.6522241478913923
Recall: 0.6249654027124274
F1-score: 0.638303886925795
--------------------------------------------------
Modelo: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True)
Precisió: 0.7036930178880554
Recall: 0.6750622751176307
F1-score: 0.6890803785845458
--------------------------------------------------
Modelo: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True, use_context_features=True)
Precisió: 0.7442737025224703
Recall: 0.7104898975920287
F1-score: 0.7269895213820448
--------------------------------------------------


Tornem a veure com el model que usa el 3 'features' obte els millors resultats i en aquesta ocasió tornarem a utilitzar el model que utilitza els 3 'features' per a identificar les entitats correctament.

In [24]:
feature_extractor = FeatureExtractor(True, True, True, model_POS=model_tagger_POS_esp)

testb_esp_bioes = convert_to_bioes(testb_esp)
testb_esp_real = obtenir_token_entity(testb_esp_bioes)

testb_esp_pre_tag = obtenir_token(testb_esp)

model_BIOES_esp = CRFTagger(feature_func=feature_extractor._get_features)
model_BIOES_esp.train(train_esp_BIOES, 'model_BIOES_esp.crf.tagger')
    
predicted_BIOES = model_BIOES_esp.tag_sents(testb_esp_pre_tag)

resultats(predicted_BIOES, testb_esp_real, obtenir_entitats_amb_posicions_bioes)

Precisió: 0.7817327065144393
Recall: 0.7673038892551087
F1-score: 0.7744510978043911


#### NED

Experimentació amb diferents 'features' per a veure quins són els més adequats pel model amb 'BIOES' i la llengua neerlandesa.

In [25]:
train_ned_bioes = convert_to_bioes(train_ned)
train_ned_BIOES = obtenir_token_entity(train_ned_bioes)

testa_ned_bioes = convert_to_bioes(testa_ned)
testa_ned_real = obtenir_token_entity(testa_ned_bioes)

def model_entrenament(train_tag, extractor, testa_pre_tag):
    if extractor == None:
        model_BIOES = CRFTagger()
        model_BIOES.train(train_tag, 'model_BIOES_ned.crf.tagger')
        
        predicted_BIOES = model_BIOES.tag_sents(testa_pre_tag)
        
    else:
        model_BIOES = CRFTagger(feature_func=extractor._get_features)
        model_BIOES.train(train_tag, 'model_BIOES_ned.crf.tagger')
        
        predicted_BIOES = model_BIOES.tag_sents(testa_pre_tag)
        
    return resultats(predicted_BIOES, testa_ned_real, obtenir_entitats_amb_posicions_bioes)

for param in param_combinations_ned: # definida primerament en la predicció del BIO
    print(f'Model: {str(param)}') if param != None else print(f'Model: Features per defecte de CRFTagger')
    model_entrenament(train_ned_BIOES, param, testa_ned_pre_tag)
    print('-'*50)

Modelo: Features per defecte de CRFTagger
Precisió: 0.0
Recall: 0.0
F1-score: 0.0
--------------------------------------------------
Modelo: FeatureExtractor()
Precisió: 0.7785087719298246
Recall: 0.16202647193062528
F1-score: 0.26822818284850775
--------------------------------------------------
Modelo: FeatureExtractor(use_basic_features=True)
Precisió: 0.6494252873563219
Recall: 0.41259698767685987
F1-score: 0.504605079542283
--------------------------------------------------
Modelo: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True)
Precisió: 0.6781725888324873
Recall: 0.6097672295755363
F1-score: 0.6421533285267964
--------------------------------------------------
Modelo: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True, use_context_features=True)
Precisió: 0.7135802469135802
Recall: 0.659516202647193
F1-score: 0.6854838709677419
--------------------------------------------------


En aquest últim model veiem com una vegada més els millors resultats ens el dona el model que usa els 3 'features' i, per tant, una vegada més utilitzarem aquest model per identificar correctament.

In [27]:
feature_extractor = FeatureExtractor(True, True, True, model_POS=model_tagger_POS_ned)

testb_ned_bioes = convert_to_bioes(testb_ned)
testb_ned_real = obtenir_token_entity(testb_ned_bioes)

testb_ned_pre_tag = obtenir_token(testb_ned)

model_BIOES_ned = CRFTagger(feature_func=feature_extractor._get_features)
model_BIOES_ned.train(train_ned_BIOES, 'model_BIOES_ned.crf.tagger')
    
predicted_BIOES = model_BIOES_ned.tag_sents(testb_ned_pre_tag)

resultats(predicted_BIOES, testb_ned_real, obtenir_entitats_amb_posicions_bioes)

Precisió: 0.7451505016722408
Recall: 0.6853275915103045
F1-score: 0.7139881429258131


## <span style="color:lightblue; font-weight:bold;">CONCLUSIONS</span> <a id="conclusions"></a>

Després de les proves realitzades, es pot observar clarament que l'ús de múltiples 'features' en un model millora significativament la identificació d'entitats. Aquesta millora és notable en comparació amb models que utilitzen menys característiques o cap, destacant la importància d'aquesta pràctica en el desenvolupament de models precisos.

Pel que fa a les codificacions BIO, IO i BIOES en les tasques d'etiquetatge d'entitats anomenades, les proves mostren que proporcionen resultats molt similars, sense destacar notablement l'una sobre les altres. Tot i això, creiem que és important assenyalar que la codificació IO tendeix a obtenir lleugerament resultats pitjors en comparació amb les altres dues, tot i que aquesta diferència no és significativa o notòria. Aquest fenomen pot atribuir-se a la menor quantitat d'informació proporcionada per la codificació IO en la detecció d'entitats, limitant així les opcions dels models predictius en la seva capacitat per detectar-les o no.

Quant a la comparació entre els models en castellà i neerlandès, és evident que els models en castellà mostren millors resultats. Aquesta diferència pot atribuir-se a diversos factors, com ara la disponibilitat de dades etiquetades i corpus de text, la complexitat lingüística i els esforços de recerca i desenvolupament en cada idioma.

En resum, és important l'ús de múltiples 'features' en els models, les codificacions 'BIO', 'IO' i 'BIOES' donen resultats molt semblants. Tot i així, es pot observar un rendiment lleugerament inferior amb la codificació 'IO' en comparació amb les altres. A més, els models entrenats en castellà demostren un millor rendiment en comparació amb els models en neerlandès.

## Execució dels models amb textos reals